# Подготовка итоговой модели и данных для предсказания ожидаемой продолжительности жизни в странах

**Оглавление**<a id='toc0_'></a>    
- [Создание таблицы с данными стран для прогноза](#toc1_)    
  - [Заполнение кодов и названий стран](#toc1_1_)    
  - [Добавление признака кластера страны](#toc1_2_)    
  - [Добавление признаков с коэффициентами стран](#toc1_3_)    
  - [one-hot кодирование кода региона](#toc1_4_)    
  - [Добавление признака со сглаженными значениями продолжительности жизни](#toc1_5_)    
  - [Заполнение остальных предикторов сглаженными значениями](#toc1_6_)    
  - [Изменение порядка колонок](#toc1_7_)    
- [Создание таблицы с данными стран за прошлые годы](#toc2_)    
- [Создание пайплайна для предсказания](#toc3_)    
- [Описание запуска проекта](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

На основе проведенных исследований и созданных признаков подготовим итоговую модель для предсказания ожидаемой продолжительности жизни в отдельных странах после 2020 г.\
Будем делать предсказание за 2021 - 2023 гг.

Возьмем за основу данные за 2000-2020 гг. и добавим признаки, которые хорошо себя показали.

Сформируем таблицы аналогично, как для итоговой модели в файле с предсказанием [../05_prediction/01_main.ipynb](../05_prediction/01_main.ipynb) .\
И сохраним модель, которая показала наилучший результат, для дальнейшего использования.

Файлы таблиц и сериализованной модели будем сохранять в папку [./app](./app/) для дальнейшего создания интерфейса.

In [55]:
import sys
# Добавим папку с корнем проекта в список системных директорий, чтобы Python видел путь к папке utils
sys.path.append('..')

In [56]:
import pandas as pd
from sklearn.preprocessing import (
    MinMaxScaler, 
    OneHotEncoder,
)
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import pickle

from utils.constants import (
    F, 
    TEST_YEARS_COUNT,
)
from utils.prepare_data import (
    get_location_codes,
    get_location_time_series,
    get_data_with_smoothing_target_feature,
)
from classes.LocationCluster import LocationCluster
from classes.LocationCoef import LocationCoef

%matplotlib inline

from warnings import simplefilter
simplefilter('ignore')

In [57]:
# Загрузим таблицу
data = pd.read_csv(
    '../data/cumulative_life_expectancy_prepared.csv'
)

data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,Sanitation,DrinkingWater,HealthCareCosts,HealthCareCostsGdp,HealthCareCostsPerCapita,GdpPerCapita,Population,Schooling,ImmunizationMean,LifeExpectancy
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,27.56,41.14,2.74,1.11,7.66,563.733796,16194869.0,5.027188,28.000000,49.37
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,28.99,42.25,5.97,2.05,13.35,533.586202,16747208.0,5.092250,43.333333,50.06
2,AFR,Africa,AGO,Angola,2002,32.51,0.01288,0.758272,29.4,0.027049,...,30.42,43.38,4.19,1.31,11.54,882.147847,17327699.0,5.157312,42.666667,51.06
3,AFR,Africa,AGO,Angola,2003,32.16,0.01265,0.676496,29.3,0.026962,...,31.86,44.36,4.26,1.47,14.55,992.698979,17943712.0,5.222375,37.000000,51.74
4,AFR,Africa,AGO,Angola,2004,32.24,0.01229,0.594192,29.0,0.026300,...,33.29,45.35,5.66,1.69,21.36,1266.210864,18600423.0,5.287437,36.000000,52.36


In [ ]:
# Загружаем таблицу с данными о странах и регионах
regions_and_locations_data = pd.read_csv(
    '../data/regions_and_locations.csv'
)

regions_and_locations_data.head()

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location
0,AFR,Africa,AGO,Angola
1,AFR,Africa,BDI,Burundi
2,AFR,Africa,BEN,Benin
3,AFR,Africa,BFA,Burkina Faso
4,AFR,Africa,BWA,Botswana


## <a id='toc1_'></a>[Создание таблицы с данными стран для прогноза](#toc0_)

Далее составим таблицу с данными стран за годы, для которых будем делать предсказание.\
Эта таблица будет нужна для создания интерфейса, с помощью которого можно будет сделать предсказание для отдельной страны и посмотреть на результат.\
Предикторы заполним сглаженными значениями на основе предыдущих лет, либо рассчитанными константами.

In [ ]:
# Определим список лет для предсказания
start_prediction_year = data[F.Period.value].max() + 1
prediction_years = list(range(start_prediction_year, start_prediction_year + TEST_YEARS_COUNT))
prediction_years

[2021, 2022, 2023]

### <a id='toc1_1_'></a>[Заполнение кодов и названий стран](#toc0_)

In [60]:
# Колонки, которые сначала заполним для будущей таблицы
future_columns = [
    F.ParentLocationCode.value,
    F.SpatialDimValueCode.value,
    F.Location.value,
    F.Period.value,
]

# Таблица-заглушка для данных за один год
future_data_dump = pd.DataFrame(columns=future_columns)
# Заполним код и название страны
copied_columns = [
    F.ParentLocationCode.value,
    F.SpatialDimValueCode.value,
    F.Location.value,
]
future_data_dump[copied_columns] = regions_and_locations_data[copied_columns]

future_data_dump.head()

,ParentLocationCode,SpatialDimValueCode,Location,Period
0,AFR,AGO,Angola,NaN
1,AFR,BDI,Burundi,NaN
2,AFR,BEN,Benin,NaN
3,AFR,BFA,Burkina Faso,NaN
4,AFR,BWA,Botswana,NaN


In [ ]:
# Заполним данные для каждого года предсказания
future_data = pd.DataFrame(columns=future_columns)

for year in prediction_years:
    future_year_data = future_data_dump.copy()
    future_year_data[F.Period.value] = year
    
    future_data = pd.concat([
        future_data,
        future_year_data,
    ], ignore_index=True)

# Убедимся, что заполнили данные за все года
future_data[F.Period.value].value_counts()

Period
2021    181
2022    181
2023    181
Name: count, dtype: int64

### <a id='toc1_2_'></a>[Добавление признака кластера страны](#toc0_)

In [ ]:
# Создадим объект класса для добавления кластера
cluster_creater = LocationCluster(2)
# Обучим его на текущих данных
cluster_creater.fit(data)

# Добавим кластер в текущие и будущие данные
data = cluster_creater.transform(data)
future_data = cluster_creater.transform(future_data)

display(data.head(2))
display(future_data.head(2))

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,DrinkingWater,HealthCareCosts,HealthCareCostsGdp,HealthCareCostsPerCapita,GdpPerCapita,Population,Schooling,ImmunizationMean,ClusterKMeans,LifeExpectancy
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,41.14,2.74,1.11,7.66,563.733796,16194869.0,5.027188,28.000000,1,49.37
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,42.25,5.97,2.05,13.35,533.586202,16747208.0,5.092250,43.333333,1,50.06


,ParentLocationCode,SpatialDimValueCode,Location,Period,ClusterKMeans
0,AFR,AGO,Angola,2021,1
1,AFR,BDI,Burundi,2021,1


### <a id='toc1_3_'></a>[Добавление признаков с коэффициентами стран](#toc0_)

In [ ]:
# Признаки, которые не будут участвовать в создании коэффициентов.
# Подробнее о причинах исключения эти признаков написано здесь ../03_eda/03_clustering.ipynb
columns_to_exclude = [
    F.Period.value,
    F.Population.value, 
    F.AlcoholСonsumption.value,
    F.BmiAdultOverweight25.value,
    F.BmiAdultOverweight30.value,
    F.BmiChildOverweight1.value,
    F.BmiTeenagerOverweight1.value,
    F.BmiChildOverweight2.value,
    F.BmiTeenagerOverweight2.value,
    F.AdultNcdMortality061.value,
    F.AdultNcdMortality080.value,#
    F.AdultNcdMortalitySum.value,
    F.AdultNcdMortality110.value,
    F.AdultNcdMortality117.value,
]

# Создаем объект класса для добавления коэффициентов
coefs_creater = LocationCoef(columns_to_exclude)
# Обучаем на текущих данных
coefs_creater.fit(data)

# Добавляем коэффициенты в текущие и будущие данные
data = coefs_creater.transform(data)
future_data = coefs_creater.transform(future_data)

display(data.head(2))
display(future_data.head(2))

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,HealthCareCostsGdp,HealthCareCostsPerCapita,GdpPerCapita,Population,Schooling,ImmunizationMean,ClusterKMeans,PositiveCoef,NegativeCoef,LifeExpectancy
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,1.11,7.66,563.733796,16194869.0,5.027188,28.000000,1,0.843237,0.874081,49.37
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,2.05,13.35,533.586202,16747208.0,5.092250,43.333333,1,0.843237,0.874081,50.06


,ParentLocationCode,SpatialDimValueCode,Location,Period,ClusterKMeans,PositiveCoef,NegativeCoef
0,AFR,AGO,Angola,2021,1,0.843237,0.874081
1,AFR,BDI,Burundi,2021,1,0.765792,0.892875


### <a id='toc1_4_'></a>[one-hot кодирование кода региона](#toc0_)

In [ ]:
# В столбец 'Reg' поместим код региона, чтобы названия итоговых столбцов были короче
data['Reg'] = data[F.ParentLocationCode.value]
future_data['Reg'] = future_data[F.ParentLocationCode.value]

# Список колонок для кодирования
columns_to_encode = ['Reg']

# Создаем объект для кодирования
one_hot_encoder = OneHotEncoder()
# Обучаем на текущих данных
one_hot_encoder.fit(data[columns_to_encode])

# Массивы с перекодированными значениями для тренировочных и тестовых данных
encoded_data = one_hot_encoder.transform(data[columns_to_encode]).toarray() 
encoded_future = one_hot_encoder.transform(future_data[columns_to_encode]).toarray() 

# Имена перекодированных колонок
encoded_column_names = one_hot_encoder.get_feature_names_out(columns_to_encode)

# Переведем перекодированные колонки в DataFrame для присоединения к таблицам
encoded_data = pd.DataFrame(encoded_data, columns=encoded_column_names)
encoded_future = pd.DataFrame(encoded_future, columns=encoded_column_names)

# Присоединим к таблицам перекодированные значения
data = pd.concat([data, encoded_data], axis=1)
future_data = pd.concat([future_data, encoded_future], axis=1)

# Удалим столбец 'Reg'
data.drop(columns=columns_to_encode, inplace=True)
future_data.drop(columns=columns_to_encode, inplace=True)

display(data.head(2))
display(future_data.head(2))

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,ClusterKMeans,PositiveCoef,NegativeCoef,LifeExpectancy,Reg_AFR,Reg_AMR,Reg_EMR,Reg_EUR,Reg_SEAR,Reg_WPR
0,AFR,Africa,AGO,Angola,2000,34.56,0.01331,0.859921,30.5,0.028126,...,1,0.843237,0.874081,49.37,1.0,0.0,0.0,0.0,0.0,0.0
1,AFR,Africa,AGO,Angola,2001,33.86,0.01310,0.799641,29.7,0.027503,...,1,0.843237,0.874081,50.06,1.0,0.0,0.0,0.0,0.0,0.0


,ParentLocationCode,SpatialDimValueCode,Location,Period,ClusterKMeans,PositiveCoef,NegativeCoef,Reg_AFR,Reg_AMR,Reg_EMR,Reg_EUR,Reg_SEAR,Reg_WPR
0,AFR,AGO,Angola,2021,1,0.843237,0.874081,1.0,0.0,0.0,0.0,0.0,0.0
1,AFR,BDI,Burundi,2021,1,0.765792,0.892875,1.0,0.0,0.0,0.0,0.0,0.0


### <a id='toc1_5_'></a>[Добавление признака со сглаженными значениями продолжительности жизни](#toc0_)

In [ ]:
data, future_data = get_data_with_smoothing_target_feature(
    data, future_data
)

display(data.head(2))
display(future_data.head(2))

,ParentLocationCode,ParentLocation,SpatialDimValueCode,Location,Period,AdultMortality,Homicides,MaternalMortality,AdultNcdMortality,AdultNcdMortality117,...,PositiveCoef,NegativeCoef,LifeExpectancy,Reg_AFR,Reg_AMR,Reg_EMR,Reg_EUR,Reg_SEAR,Reg_WPR,SmoothingLifeExpectancy
0,EMR,Eastern Mediterranean,AFG,Afghanistan,2000,37.82,0.00978,1.346144,43.2,0.030243,...,0.787832,0.872589,53.82,0.0,0.0,1.0,0.0,0.0,0.0,53.696819
1,EMR,Eastern Mediterranean,AFG,Afghanistan,2001,38.03,0.00964,1.273431,43.5,0.031152,...,0.787832,0.872589,53.91,0.0,0.0,1.0,0.0,0.0,0.0,54.498708


,ParentLocationCode,SpatialDimValueCode,Location,Period,ClusterKMeans,PositiveCoef,NegativeCoef,Reg_AFR,Reg_AMR,Reg_EMR,Reg_EUR,Reg_SEAR,Reg_WPR,SmoothingLifeExpectancy
0,EMR,AFG,Afghanistan,2021,1,0.787832,0.872589,0.0,0.0,1.0,0.0,0.0,0.0,60.501268
1,EMR,AFG,Afghanistan,2022,1,0.787832,0.872589,0.0,0.0,1.0,0.0,0.0,0.0,60.399577


### <a id='toc1_6_'></a>[Заполнение остальных предикторов сглаженными значениями](#toc0_)

Напишем функцию, которая заполнит сглаженными значениями предикторы.

In [66]:
def fill_field_with_smoothed_value(
    source_data: pd.DataFrame,
    future_data: pd.DataFrame,
    source_field_name: str,
    smoothing_field_name: str,
) -> pd.DataFrame:
    """Формирует столбец с прогнозом будущих значений

    Args:
        source_data (pd.DataFrame): исходная таблица с текущими данными
        future_data (pd.DataFrame): таблица с предполагаемыми будущими значениями, 
             которые получим путем сглаживания значений из текущих данных
        source_field_name (str): имя поля в исходной таблицы, 
            значения которого будем прогнозировать на будущее путем сглаживания
        smoothing_field_name (str): имя поля со сглаженным спрогнозированным значением

    Returns:
        pd.DataFrame: таблица future_data с добавленным в нее признаком smoothing_field_name
    """
    # Копируем таблицы, чтобы не мутировать их    
    source_data = source_data.copy()
    future_data = future_data.copy()
    
    # Поля для сортировки
    fields_for_sorting = [F.SpatialDimValueCode.value, F.Period.value]
    
    # На всякий случай сортируем таблицы, хотя они должны приходить отсортированные
    source_data.sort_values(by=fields_for_sorting, inplace=True)
    future_data.sort_values(by=fields_for_sorting, inplace=True)
    
    # Коды всех стран
    location_codes = get_location_codes(source_data)
    
    # Список с годами, для которых будем считать сглаженные значения
    years_future_list = sorted(list(future_data[F.Period.value].unique()))
    # Количество предсказаний, которое будем делать
    predictions_count = len(years_future_list)
    
    # Список столбцов для таблицы со сглаженными значениями
    smoothing_columns = [F.SpatialDimValueCode.value, F.Period.value, smoothing_field_name]
    # Таблица, в которую будем собирать сглаженные значения
    smoothing_data = pd.DataFrame(columns=smoothing_columns)
    
    # Для каждой страны посчитаем сглаженные значения
    for code in location_codes:
        # Временной ряд значений признака исходной таблицы
        location_time_series = get_location_time_series(source_data, code, source_field_name)
        
        # Получим объект с моделью
        exp_smoothing_model = ExponentialSmoothing(
            location_time_series, # тренировочные данные
            trend='add', # тип тренда - аддитивный
            damped_trend=True, # затухание тренда
        )
        
        # Обучим модель
        exp_smoothing_model_fit = exp_smoothing_model.fit(
            smoothing_level=0.85, # альфа - коэффициент сглаживания для уровня
            smoothing_trend=0.5, # beta - коэффициент сглаживания тренда
        )
        # Получим прогноз 
        forecast = exp_smoothing_model_fit.forecast(predictions_count)
        
        # Сформируем таблицу с полученным для страны прогнозом
        location_smoothing_data = pd.DataFrame({
            F.SpatialDimValueCode.value: code,
            F.Period.value: years_future_list,
            smoothing_field_name: forecast.values,
        })
        
        # Добавим значения прогноза, полученного для страны, в общую таблицу
        smoothing_data = pd.concat(
            [smoothing_data, location_smoothing_data],
            ignore_index=True
        )
    
    # Добавим столбец с полученными сглаженными значениями
    future_data = future_data.merge(
        smoothing_data,
        on=[F.SpatialDimValueCode.value, F.Period.value],
        how='left',
    )
    # Переведем столбцы в числовой тип
    future_data[F.Period.value] = future_data[F.Period.value].astype(int)
    future_data[smoothing_field_name] = future_data[smoothing_field_name].astype(float)
    
    return future_data

Заполним предикторы прогнозом со сглаженными значениями.

In [ ]:
future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.ImmunizationMean.value,
    F.ImmunizationMean.value,
)

future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.GdpPerCapita.value,
    F.GdpPerCapita.value,
)

future_data = fill_field_with_smoothed_value(
    data,
    future_data,
    F.Sanitation.value,
    F.Sanitation.value,
)

future_data.head()

,ParentLocationCode,SpatialDimValueCode,Location,Period,ClusterKMeans,PositiveCoef,NegativeCoef,Reg_AFR,Reg_AMR,Reg_EMR,Reg_EUR,Reg_SEAR,Reg_WPR,SmoothingLifeExpectancy,ImmunizationMean,GdpPerCapita,Sanitation
0,EMR,AFG,Afghanistan,2021,1,0.787832,0.872589,0.0,0.0,1.0,0.0,0.0,0.0,60.501268,61.287843,509.819659,54.357740
1,EMR,AFG,Afghanistan,2022,1,0.787832,0.872589,0.0,0.0,1.0,0.0,0.0,0.0,60.399577,60.222715,512.004269,56.044841
2,EMR,AFG,Afghanistan,2023,1,0.787832,0.872589,0.0,0.0,1.0,0.0,0.0,0.0,60.311459,59.370612,513.794197,57.723506
3,AFR,AGO,Angola,2021,1,0.843237,0.874081,1.0,0.0,0.0,0.0,0.0,0.0,62.838936,47.488205,1244.502607,52.225953
4,AFR,AGO,Angola,2022,1,0.843237,0.874081,1.0,0.0,0.0,0.0,0.0,0.0,63.011744,45.747820,997.910059,52.681321


### <a id='toc1_7_'></a>[Изменение порядка колонок](#toc0_)

In [68]:
predictor_features = [
    F.Sanitation.value,
    F.GdpPerCapita.value,
    F.PositiveCoef.value,
    F.NegativeCoef.value,
    F.ImmunizationMean.value,
    F.SmoothingLifeExpectancy.value,
    'Reg_AFR',
    'Reg_EUR',
]

future_data = future_data[
    [F.SpatialDimValueCode.value, F.Location.value, F.Period.value] 
    + predictor_features
]

display(future_data.head(2))

,SpatialDimValueCode,Location,Period,Sanitation,GdpPerCapita,PositiveCoef,NegativeCoef,ImmunizationMean,SmoothingLifeExpectancy,Reg_AFR,Reg_EUR
0,AFG,Afghanistan,2021,54.357740,509.819659,0.787832,0.872589,61.287843,60.501268,0.0,0.0
1,AFG,Afghanistan,2022,56.044841,512.004269,0.787832,0.872589,60.222715,60.399577,0.0,0.0


In [ ]:
# Сохраним полученную таблицу для дальнейшего использования
future_data.to_csv('./app/data/future_data.csv', index=False)

## <a id='toc2_'></a>[Создание таблицы с данными стран за прошлые годы](#toc0_)

Эта таблица будет нужна для построения графика значений целевой переменной за прошлые года.

In [70]:
past_data = data[[
    F.SpatialDimValueCode.value,
    F.Location.value,
    F.Period.value,
    F.LifeExpectancy.value,
]]
past_data.head()

,SpatialDimValueCode,Location,Period,LifeExpectancy
0,AFG,Afghanistan,2000,53.82
1,AFG,Afghanistan,2001,53.91
2,AFG,Afghanistan,2002,55.15
3,AFG,Afghanistan,2003,56.09
4,AFG,Afghanistan,2004,56.48


In [ ]:
# Сохраним таблицу
past_data.to_csv('./app/data/past_data.csv', index=False)

## <a id='toc3_'></a>[Создание пайплайна для предсказания](#toc0_)

Для пайплайна возьмем модель, которая показала лучшие результаты при исследовании.

In [72]:
# Создаём пайплайн
pipe = Pipeline([  
  ('MinMaxScaler', MinMaxScaler()),
  ('LinearRegression', LinearRegression()),
])

# Обучаем пайплайн
pipe.fit(data[predictor_features], data[F.LifeExpectancy.value])

# Сериализуем pipeline и записываем результат в файл
with open('./app/model/pipeline.pkl', 'wb') as output:
    pickle.dump(pipe, output)

Проверим, на корректность сохраненный пайплайн.

In [73]:
# Десериализуем pipeline из файла
with open('./app/model/pipeline.pkl', 'rb') as pkl_file:
    pipe_from_file = pickle.load(pkl_file)

pipe_from_file

Pipeline(steps=[('MinMaxScaler', MinMaxScaler()),
                ('LinearRegression', LinearRegression())])

In [ ]:
# Сравниваем предсказания исходного и восстановленного пайплайнов
all(pipe.predict(future_data[predictor_features]) == pipe_from_file.predict(future_data[predictor_features]))

True

## <a id='toc4_'></a>[Описание запуска проекта](#toc0_)

В папке [./app](./app/) находятся исходные файлы созданного интерфейса для взаимодействия с моделью.

Данный функционал помещен также в docker https://hub.docker.com/repository/docker/experiment0/life_expectancy/general

Для запуска необходимо выполнить команды:

`docker pull experiment0/life_expectancy`

`docker run -it --rm --name=life_expectancy_container -p=5000:5000 experiment0/life_expectancy`

Далее открыть браузер по ссылке http://localhost:5000/⁠